In [1]:
import cv2
from time import time
import numpy as np
import pandas as pd
import os
import mediapipe as mp
import math
import matplotlib.pyplot as plt

In [2]:
# Initializing mediapipe pose class.
mp_pose = mp.solutions.pose

# Setting up the Pose function.
# pose = mp_pose.Pose(min_detection_confidence=0.3, model_complexity=2)

# Initializing mediapipe drawing class, useful for annotation.
mp_drawing = mp.solutions.drawing_utils

In [3]:
def detectPose(image, pose, display=True):
    '''
    This function performs pose detection on an image.
    Args:
        image: The input image with a prominent person whose pose landmarks needs to be detected.
        pose: The pose setup function required to perform the pose detection.
        display: A boolean value that is if set to true the function displays the original input image, the resultant image, 
                 and the pose landmarks in 3D plot and returns nothing.
    Returns:
        output_image: The input image with the detected pose landmarks drawn.
        landmarks: A list of detected landmarks converted into their original scale.
    '''
    
    # Create a copy of the input image.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Pose Detection.
    results = pose.process(imageRGB)
    
    # Retrieve the height and width of the input image.
    height, width, _ = image.shape
    
    # Initialize a list to store the detected landmarks.
    landmarks = []
    
    # Check if any landmarks are detected.
    if results.pose_landmarks:
    
        # Draw Pose landmarks on the output image.
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)
        
        # Iterate over the detected landmarks.
        for landmark in results.pose_landmarks.landmark:
            
            # Append the landmark into the list.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                                  (landmark.z * width)))
    
    # Check if the original input image and the resultant image are specified to be displayed.
    if display:
    
        # Display the original input image and the resultant image.
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
        # Also Plot the Pose landmarks in 3D.
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
        
    # Otherwise
    else:
        
        # Return the output image and the found landmarks.
        return output_image, landmarks


In [4]:
def calculateAngle(landmark1, landmark2, landmark3):
    '''
    This function calculates angle between three different landmarks.
    Args:
        landmark1: The first landmark containing the x,y and z coordinates.
        landmark2: The second landmark containing the x,y and z coordinates.
        landmark3: The third landmark containing the x,y and z coordinates.
    Returns:
        angle: The calculated angle between the three landmarks.

    '''

    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # Check if the angle is less than zero.
    if angle <0:

        # Add 360 to the found angle.
        angle += 360
    if angle > 180 : 
        angle = 360-angle
    
    # Return the calculated angle.
    return angle

In [5]:

# Calculate the angle between the three landmarks.
angle = calculateAngle((558, 326, 0), (642, 333, 0), (718, 321, 0))
 
# Display the calculated angle.
print(f'The calculated angle is {angle}')

The calculated angle is 166.26373169437744


In [6]:
def classifyPose(landmarks, output_image, display=False):
    '''
    This function classifies yoga poses depending upon the angles of various body joints.
    Args:
        landmarks: A list of detected landmarks of the person whose pose needs to be classified.
        output_image: A image of the person with the detected pose landmarks drawn.
        display: A boolean value that is if set to true the function displays the resultant image with the pose label 
        written on it and returns nothing.
    Returns:
        output_image: The image with the detected pose landmarks drawn and pose label written.
        label: The classified pose label of the person in the output_image.

    '''
    
    # Initialize the label of the pose. It is not known at this stage.
    label = 'Unknown Pose'

    # Specify the color (Red) with which the label will be written on the image.
    color = (0, 0, 255)
    
    # Calculate the required angles.
    #----------------------------------------------------------------------------------------------------------------
    
    # Get the angle between the left shoulder, elbow and wrist points. 
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    
    # Get the angle between the right shoulder, elbow and wrist points. 
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])   
    
    # Get the angle between the left elbow, shoulder and hip points. 
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])

    # Get the angle between the right hip, shoulder and elbow points. 
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])

    # Get the angle between the left hip, knee and ankle points. 
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    # Get the angle between the right hip, knee and ankle points 
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the warrior II pose or the T pose.
    # As for both of them, both arms should be straight and shoulders should be at the specific angle.
    #----------------------------------------------------------------------------------------------------------------
    
#     # Check if the both arms are straight.
#     if left_elbow_angle &gt; 165 and left_elbow_angle &lt; 195 and right_elbow_angle &gt; 165 and right_elbow_angle &lt; 195:

#         # Check if shoulders are at the required angle.
#         if left_shoulder_angle &gt; 80 and left_shoulder_angle &lt; 110 and right_shoulder_angle &gt; 80 and right_shoulder_angle &lt; 110:

#     # Check if it is the warrior II pose.
#     #----------------------------------------------------------------------------------------------------------------

#             # Check if one leg is straight.
#             if left_knee_angle &gt; 165 and left_knee_angle &lt; 195 or right_knee_angle &gt; 165 and right_knee_angle &lt; 195:

#                 # Check if the other leg is bended at the required angle.
#                 if left_knee_angle &gt; 90 and left_knee_angle &lt; 120 or right_knee_angle &gt; 90 and right_knee_angle &lt; 120:

#                     # Specify the label of the pose that is Warrior II pose.
#                     label = 'Warrior II Pose' 
                        
#     #----------------------------------------------------------------------------------------------------------------
    
#     # Check if it is the T pose.
#     #----------------------------------------------------------------------------------------------------------------
    
#             # Check if both legs are straight
#             if left_knee_angle &gt; 160 and left_knee_angle &lt; 195 and right_knee_angle &gt; 160 and right_knee_angle &lt; 195:

#                 # Specify the label of the pose that is tree pose.
#                 label = 'T Pose'

#     #----------------------------------------------------------------------------------------------------------------
    
#     # Check if it is the tree pose.
#     #----------------------------------------------------------------------------------------------------------------
    
#     # Check if one leg is straight
#     if left_knee_angle &gt; 165 and left_knee_angle &lt; 195 or right_knee_angle &gt; 165 and right_knee_angle &lt; 195:

#         # Check if the other leg is bended at the required angle.
#         if left_knee_angle &gt; 315 and left_knee_angle &lt; 335 or right_knee_angle &gt; 25 and right_knee_angle &lt; 45:

#             # Specify the label of the pose that is tree pose.
#             label = 'Tree Pose'
#    check if left arm is up
    if left_shoulder_angle > 150 and left_shoulder_angle < 195 and right_shoulder_angle < 110:
        label = 'left arm up'
        
#     check if righ tis up
    if left_shoulder_angle < 110 and right_shoulder_angle > 150 and right_shoulder_angle < 195:
        label = 'right arm up'
        
        
    print("el "+ str(right_elbow_angle))
    print(right_shoulder_angle)
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if the pose is classified successfully
    if label != 'Unknown Pose':
        
        # Update the color (to green) with which the label will be written on the image.
        color = (0, 255, 0)  
    
    # Write the label on the output image. 
    cv2.putText(output_image, label, (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
    
    # Check if the resultant image is specified to be displayed.
    if display:
    
        # Display the resultant image.
        print(label)
#         cv2.imshow(output_image)
#         plt.figure(figsize=[10,10])
#         plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
    else:
        
        # Return the output image and the classified label.
        return output_image, label

In [7]:
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)
 
# Initialize the VideoCapture object to read from the webcam.
video = cv2.VideoCapture(0)
 
# Initialize the VideoCapture object to read from a video stored in the disk.
# video = cv2.VideoCapture('media/running.mp4')
 
 
# Initialize a variable to store the time of the previous frame.
time1 = 0
while (video.isOpened() == False):
    print("Error opening the video file")
# Iterate until the video is accessed successfully.
while True:
    
    # Read a frame.
    ok, frame = video.read()
    
    # Check if frame is not read properly.
    if not ok:
        
        # Break the loop.
        break
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Get the width and height of the frame
    frame_height, frame_width, _ =  frame.shape
    
    # Resize the frame while keeping the aspect ratio.
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
    
    # Perform Pose landmark detection.
    frame, landmarks = detectPose(frame, pose_video, display=False)
    if landmarks:
        classifyPose(landmarks,frame,display = False)
    
    # Set the time for this frame to the current time.
    time2 = time()
    
    # Check if the difference between the previous and this frame time &gt; 0 to avoid division by zero.
    if (time2 - time1) > 0:
    
        # Calculate the number of frames per second.
        frames_per_second = 1.0 / (time2 - time1)
        
        # Write the calculated number of frames per second on the frame. 
        cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (40, 50),cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    
    # Update the previous frame time to this frame time.
    # As this frame will become previous frame in next iteration.
    time1 = time2
    
    # Display the frame.
    cv2.imshow('Pose Detection', frame)
    
    # Wait until a key is pressed.
    # Retreive the ASCII code of the key pressed
    k = cv2.waitKey(1) &0xFF
    
    # Check if 'ESC' is pressed.
    if(k == 27):
        
        # Break the loop.
        break
 
# Release the VideoCapture object.
video.release()
 
# Close the windows.
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


el 157.47091146537707
26.78360218307304
el 153.1711399697734
28.490994354070487
el 152.40407216565163
29.23919482875345
el 156.20921231504747
30.44062516679099
el 151.84625897425235
30.584963178427465
el 153.662410211578
31.771749664441526
el 155.72496039991273
31.935959786165956
el 156.54856498434495
31.880334859368343
el 157.56371128242878
31.698020858673235
el 154.89162001799335
31.142500390606564
el 155.3844341538118
31.06514599602454
el 157.53171664843933
31.24843369770784
el 158.09620225944212
31.00314409198703
el 158.93282984493777
31.390295581734648
el 158.85179746368115
31.332870069977606
el 158.8180524592052
31.230449186625
el 158.48187312559767
31.24860209887138
el 159.26856120995313
30.97813452603668
el 162.13918914568703
31.245424384473324
el 162.38795014027266
30.958974311943315
el 162.48940073961433
28.177305441624505
el 162.06034894893097
28.67293131014755
el 157.1089507040223
27.65770766545182
el 156.99451483640578
28.228346146425384
el 155.25187936762913
29.7358421474

el 154.15981567198213
30.455392349500315
el 159.06965463589648
28.42590705060648
el 162.5610016939155
26.401773610753978
el 169.66198578551024
24.63479460408244
el 176.22888895436293
18.235995931922332
el 160.7270490543787
15.757849010972436
el 155.8505295004153
23.075350588350453
el 175.41714894792312
18.67060149965099
el 148.05320695085194
11.994504635624708
el 60.763956907593595
7.486243883592522
el 147.257265763467
7.6208199491784585
el 151.77491181151967
7.003184011749826
el 177.41266215398184
33.80451587216609
el 105.95124678723036
29.903472679756476
el 171.69584433030138
49.66234742553539
el 172.43096069682875
41.48546923257007
el 178.16970098013687
23.506427961420798
el 159.77514056883192
24.11355589113777
el 179.628952516795
30.05359226603141
el 178.55869835376717
29.268669684995505
el 178.9425208999891
30.232373965112227
el 177.84526485672757
28.10022129698075
el 176.58500416858345
25.761355483091553
el 177.61699569846164
24.375999084761087
el 175.4703556041578
21.49458354813

el 177.4443347762466
18.14196971522655
el 176.86812038373077
18.506095665767138
el 176.89158128418006
18.64342254032159
el 177.87461381975976
18.081160725693437
el 178.3788540863466
18.292929183959046
el 179.97043997486682
17.37096071865416
el 179.90632386844973
17.307985955475267
el 179.21724655925217
17.101170832984206
el 179.21724655925217
17.101170832984206
el 179.76259409463182
17.687825893055273
el 179.76259409463182
17.687825893055273
el 178.23788402589653
16.95744279242147
el 179.68080675892546
17.91658710034423
el 179.84020431402996
17.357450256332957
el 179.84731631368487
17.248790523200018
el 179.84731631368487
17.248790523200018
el 179.84731631368487
17.039883406207966
el 179.33344545760548
16.828870634564737
el 179.6045902697762
17.41508051459395
el 179.0486978210326
17.757082388476274
el 178.56834550052412
17.98476127643287
el 177.09662800558309
18.941251345531022
el 177.58965517538212
18.7310690864561
el 177.11594635918038
19.41434775345207
el 177.27451441529215
19.73908

el 177.1413999061764
20.173904236216448
el 175.8882066394381
20.531220938262358
el 172.9980710060022
20.26491063420571
el 174.02930432971738
19.764960151197666
el 170.07730908670757
20.473204091659344
el 166.2701481543442
20.173965689538527
el 163.39405369101402
20.433674968149294
el 158.74252752847582
20.31713682521638
el 155.53549661013872
21.082787853256146
el 153.77912612001336
22.07568801876312
el 153.19079214947894
22.844553127664927
el 153.86573603993065
22.90605442940078
el 153.77456856584175
23.95394050376169
el 156.80424240499045
23.319495014049465
el 156.9823758673013
24.240340119144268
el 156.89979939452658
25.13029915220729
el 158.93038803366528
23.93430310058381
el 156.6751617732971
25.346290849962386
el 150.22038117755864
28.713455839193607
el 148.29006467759024
28.4727227580706
el 144.1814125383742
29.193063087489403
el 137.6468098856272
29.20367026240413
el 132.96644859043383
29.62583558655176
el 124.76066826513102
29.180853749417732
el 125.75539344903646
29.6342586388

...

el 84.91212583279133
4.483369030290701
el 84.79004237834971
4.114707039842259
el 84.37789143605517
4.648941436578433
el 85.18913328018812
5.573500859345908
el 85.08127954292178
5.573500859345908
el 85.08127954292178
5.573500859345908
el 83.71582579687106
5.033233365587957
el 83.71582579687106
5.033233365587957
el 84.12439491789848
4.494031138694421
el 84.12439491789848
4.494031138694421
el 84.25358058459432
4.326015821191069
el 84.25358058459432
4.326015821191069
el 84.46901686824742
4.864060719415161
el 84.46901686824742
4.494031138694421
el 83.99099404250546
4.494031138694421
el 84.52903894072955
4.326015821191069
el 83.71552830435184
4.021907543683611
el 83.71552830435184
4.021907543683611
el 83.79319581961283
4.0866147643422455
el 84.00354979361043
4.614876262982496
el 83.87014891821741
4.614876262982496
el 84.27515979273124
4.55501651163275
el 83.71552830435184
4.021907543683611
el 83.71552830435184
4.021907543683611
el 83.48757697071215
3.4900124517127784
el 82.79929574332397
2.9

el 113.96591200232558
23.95256691663542
el 114.455042676457
23.95256691663542
el 114.20147765566085
23.834974561604554
el 116.13100054493856
22.9662830602697
el 118.49430974312895
22.285888599549658
el 119.54544378487546
21.68067400230484
el 122.1608548228271
21.178972468604414
el 124.19047373616135
20.405313323877696
el 124.19047373616135
20.405313323877696
el 124.19047373616135
20.405313323877696
el 125.11201118442219
19.895968617019143
el 124.19047373616135
20.405313323877696
el 125.61836127262944
19.74414293205357
el 125.61836127262944
19.74414293205357
el 125.61836127262944
19.80782847030898
el 125.5302211259193
19.96046665301673
el 125.5302211259193
19.96046665301673
el 125.44519073830403
20.103985931776066
el 126.36646280710369
19.96046665301673
el 125.8571181002452
20.103985931776066
el 125.76297461034994
20.198129421671307
el 125.76297461034994
20.13588265243966
el 125.76297461034994
20.13588265243966
el 125.8571181002452
19.98027824596893
el 126.26294884654644
20.041739162544

el 175.99287840799403
26.47278752170138
el 174.63643903696885
25.51080856878767
el 175.54963029515392
25.294511592096136
el 173.91620762126314
24.667671170280137
el 173.68956070085792
24.443954780416536
el 173.35644945693656
24.823670882126706
el 176.76627602936617
27.332256958744324
el 177.31159857541854
28.212079838336738
el 178.30799018730997
29.00668477143245
el 178.7944968448551
30.086833809407537
el 177.142876951389
29.54875125056881
el 175.75513794870102
29.884610199093533
el 176.05773846746905
29.884610199093533
el 176.92666748892663
29.884610199093533
el 176.38460100979032
29.67781426090788
el 175.75513794870102
29.884610199093533
el 175.84837601495414
29.625305480790953
el 175.5457754961861
29.625305480790953
el 175.75513794870102
29.884610199093533
el 174.26350461312884
29.342543719957174
el 174.37328901363634
29.13751931307558
el 174.0859636431331
29.085879084788168
el 175.5110493546321
29.615545671219394
el 175.5110493546321
29.615545671219394
el 175.5110493546321
29.61554

el 178.02104597048475
18.193648417253367
el 177.24457529949726
17.251128348687967
el 176.43009068664827
16.68157229950459
el 175.01767465160626
16.22740804359795
el 175.07539287281176
15.74751943608113
el 175.39328207538514
15.768328217972053
el 175.78695088653956
15.953517969504237
el 175.82150500894784
15.220962096377697
el 178.1792848671553
15.706993820924797
el 179.54772190201604
15.932895429320862
el 178.2701163781869
16.158591824783603
el 176.75479522255455
16.521259639643663
el 171.2882071653602
16.89091980176658
el 172.0733551432615
16.670962438691852
el 172.3244157731956
16.885985430835095
el 174.1610285852502
17.2456341575982
el 174.80159649690455
17.34283395511838
el 175.5469014987797
18.69089054557663
el 174.9110174471394
18.982462746021813
el 175.28621950727396
19.181860690957137
el 176.4556027051598
18.880940076462352
el 178.0196791910976
18.51011108027126
el 177.37235454666006
19.31010183032545
el 175.71039203297815
18.676701732190104
el 176.13532772395655
18.74581602859

el 147.92983995571186
33.78756450421809
el 157.08832821792242
34.3595538223072
el 155.3318981442667
34.23303921365801
el 159.84109434259923
34.28619637058687
el 159.31826859808234
34.207030635855645
el 154.8705505054891
34.78641835469969
el 156.43494115201588
34.669599988157884
el 156.2599089353872
35.93138960669477
el 160.5625093628026
31.755831427405216
el 170.02992890332973
30.910981803817815
el 171.17404132051632
32.75652415770436
el 170.6858631068892
32.43148478841356
el 171.85029289438057
30.217071756486686
el 174.8785544446797
27.559606598589383
el 173.40555341422882
25.80799889036366
